# i.MXRT118x Signed AHAB Example

This example demonstrates how to create a signed AHAB container for i.MXRT118x devices. The aim of this Jupyter notebook is to show how to create a signed AHAB container for an application based on the SDK. The `nxpmemcfg` tool is demonstrated to generate the FCB block for the external memory populated on the RT1180-EVK board. The resulting image must be programmed into the external memory via the `blhost` tool.

## 1. Prerequisites
- SPSDK is needed with examples extension. `pip install spsdk[examples]` (Please refer to the [installation](../../_knowledge_base/installation_guide.rst) documentation.)

- This example uses RT1180 EVK board

In [1]:
import os

from spsdk.utils.jupyter_utils import YamlDiffWidget

# This env variable sets colored logger output to STDOUT
%env JUPYTER_SPSDK=1
# Set a magic for command execution and echo
%alias execute echo %l && %l
%alias_magic ! execute

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


In [2]:
WORKSPACE = "workspace/"  # change this to path to your workspace
INPUTS = "inputs/"
VERBOSITY = (
    ""  # verbosity of commands, might be -v or -vv for debug or blank for no additional info
)
BIMG_FILE = (
    INPUTS + "bootimg_rt118x_flexspi_nor.yaml"
)  # Boot image yaml file. Used to generate bootable image

SRKH_BATCH_FILE = (
    WORKSPACE + "signed_ahab_oem0_srk0_hash_blhost.bcf"
)  # batch file for SRKH programming
BOOTABLE_IMAGE = WORKSPACE + "bootable_img.bin"  # Final Bootable image contains SIGNED_AHAB + FCB

FLASHLOADER_FILE = "../rt118x_signed_flashloader/workspace/flashloader.bin"  # Path to Flashloader
CFG_MEM_FILE = WORKSPACE + "config_mem.bls"  # Configure memory template
FCB_FILE = WORKSPACE + "fcb.bin"  # Firmware Configuration block file

### 1.1 Compile an Application
Hello_word XiP application is used in this example. 

To use a custom application follow the steps:

- Download the NXP MCUXpresso SDK from NXP web site (https://mcuxpresso.nxp.com/)
- Open the example in your favorite IDE. 
- Compile your application using the corresponding target. The resulting binary must be copied at input folder and renamed in regards to specific target XiP FlexSPI1/2, CTCM, OCRAM, Multicore. to make it running with this example. 
    - fspi1_xip_cm33_img.bin (default)
    - fspi2_xip_cm33_img.bin
    - ctcm_cm33_img.bin
    - ocram_cm33_img.yaml
    - fspi1_multicore_img.bin
    - fspi2_multicore_img.bin


## 2. Generate Keys
The AHAB is using asymmetric algorithm for image authentication which requires keys generated according to PKI for operation. The example uses pre-generated ECC-256 keys. To generate your own keys please refer to [How-to-get-keys-using-nxpcrypto](../../crypto/keys.ipynb) example.

## 3. Sign NXP Flashloader
On secured device all SW executed on the device must be authenticated, including NXP Flashloader. The example uses pre-generated ECC-256 keys to sign the Flashloader. To generate your own keys please refer to the [Flashloader](../rt118x_signed_flashloader/rt118x_signed_flashloader.ipynb) example:

## 4. Create Signed AHAB Container for Application

In the previous steps, we generated a signed AHAB container for the NXP Flashloader. We need to perform similar steps for the application that will run on secured devices. The SPSDK tool, `nxpimage`, is specifically designed to create bootable images, including AHAB containers for the RT118x family. The `ahab` subcommand is used to generate an AHAB container for an application.

A YAML file for the RT118x can be generated using the following command. The steps below guide you on which fields need to be reconfigured. The resulting YAML file is available in the input folder, so manual updates are not necessary.


In [3]:
YamlDiffWidget("inputs/rt118x_secure_boot.diffc").html

nxpimage ahab get-template -f mimxrt1189 -o workspace/ahab_template.yaml --force 
Creating workspace/ahab_template.yaml template file.


### 4.1 Modify the Template Configuration File for AHAB

To create an AHAB container for your application, follow these steps to modify the template configuration file:

I. **Change the Output File Name**: The target memory is standard by default, representing the memory from which the RT118x will boot.

II. **General Settings of AHAB Container**:
    - Remove the signature provider definition.
    - Configure the path to the pre-generated private signing key.
    - Set the `use_srk_id` field to align with the `signer` index:
      - `use_srk_id=0` corresponds to `srk0_ecc256.pem`
      - `use_srk_id=1` corresponds to `srk1_ecc256.pem`
      - `use_srk_id=2` corresponds to `srk2_ecc256.pem`
      - `use_srk_id=3` corresponds to `srk3_ecc256.pem`

III. **Define the Image Executable**:
    - Set the application offset to `0xA000` to align with the SDK linker files. This offset allows for the inclusion of the base ELE firmware (32KB).
    - Ensure `load_address` and `entry_point` fields match the start address defined in the linker file for the application. Both usually point to the same address unless there are data placed before the image vector table (e.g., dual-core applications).
    - The `hash` parameter is mandatory for AHAB. Integrity checks are always performed during boot:
      - In OEM_OPEN, an invalid hash reports a warning but allows the image to boot.
      - In OEM_CLOSED, an invalid hash prevents the firmware from booting.

  - The example below is for an XiP target executed from NOR memory connected to FlexSPI 1. Examples for other bootable images (FlexSPI2, OCRAM, CTCM, and multicore) are also available in the inputs folder:
    - `config_signed_fspi1.yaml` (demonstrated)
    - `config_signed_fspi2.yaml`
    - `config_signed_ctcm.yaml`
    - `config_signed_ocram.yaml`
    - `config_signed_multicore_fspi1.yaml`
    - `config_signed_multicore_fspi2.yaml`

IV. **Define the SRK (Super Root Keys) Array**:
    - Use local pre-generated public keys.

V. **Remove Unused Parts**:
    - Remove unused sections for AHAB Certificate and Encryption blob from the YAML file.

To export AHAB container is designed SPSDK tool in 'nxpimage' under 'ahab' sub commands group.

In [4]:
# For this example, signed xip application from FlexSPI 1 is used by default
AHAB_CONFIG = INPUTS + "config_signed_fspi1.yaml"  # Prepared AHAB configuration file
# AHAB_CONFIG = INPUTS + "config_signed_fspi2.yaml"                   # Prepared AHAB configuration file
# AHAB_CONFIG = INPUTS + "config_signed_ctcm.yaml"                    # Prepared AHAB configuration file
# AHAB_CONFIG = INPUTS + "config_signed_ocram.yaml"                   # Prepared AHAB configuration file
# AHAB_CONFIG = INPUTS + "config_signed_multicore_fspi1.yaml"         # Prepared AHAB configuration file
# AHAB_CONFIG = INPUTS + "config_signed_multicore_fspi2.yaml"         # Prepared AHAB configuration file

assert os.path.exists(AHAB_CONFIG)
# Export AHAB container using nxpimage
%! nxpimage $VERBOSITY ahab export -c $AHAB_CONFIG

nxpimage  ahab export -c inputs/config_signed_fspi1.yaml 
Success. (AHAB: workspace/signed_ahab.bin created.)
Exporting AHAB fuses to the output directory.
workspace/ahab_oem0_srk0_hash.txt
workspace/ahab_oem0_srk0_hash_blhost.bcf


## 5. Program the image into external memory.

RT118x requires boot from an external memory. To build a valid bootable image, another structures must be placed into external memory depending on bootable image type. For signed AHAB, ony FCB is required besides ahab and image itself. See the figure below for information:



![Boot image structure](img/boot_image-1.png "boot image")




The SPSDK contains commands group that simplify whole operation with bootable images under 'nxpimage'. 


I. Configure external memory. SPDSK contains tool called "nxpmemcfg" which contains database of supported memories and utilities to supports creation, extraction, parsing of FCB etc.

In [5]:
# Print out all supported memories
%! nxpmemcfg family-info

nxpmemcfg family-info 
List of all supported peripherals and its instances:
╔════╦══════════════════════════╦═════════════╦══════════╦══════════════╦══════════╦═══════════╦════════════════════════════════════════════════╦════════╦════════╗
║ #  ║          Family          ║ flexspi_nor ║ xspi_nor ║ flexspi_nand ║ semc_nor ║ semc_nand ║                    spi_nor                     ║  mmc   ║   sd   ║
╠════╬══════════════════════════╬═════════════╬══════════╬══════════════╬══════════╬═══════════╬════════════════════════════════════════════════╬════════╬════════╣
║ 0  ║  lpc5502, Revision: a0   ║     N/A     ║   N/A    ║     N/A      ║   N/A    ║    N/A    ║                      [3]                       ║  N/A   ║  N/A   ║
║ 1  ║  lpc5504, Revision: a0   ║     N/A     ║   N/A    ║     N/A      ║   N/A    ║    N/A    ║                      [3]                       ║  N/A   ║  N/A   ║
║ 2  ║  lpc5506, Revision: a0   ║     N/A     ║   N/A    ║     N/A      ║   N/A    ║    N/A    ║        

In [6]:
# How to use the blhost-script
%! nxpmemcfg blhost-script

nxpmemcfg blhost-script 

Help for nested command: 'blhost-script'
Command Hierarchy: nxpmemcfg ▶ blhost-script

────────────────────── Root command (nxpmemcfg) help ──────────────────────

Options:
  -v, --verbose  Print more detailed information
  -vv, --debug   Display more debugging information.
  --version      Show the version and exit.
  --help         Show this message and exit.

─────────────────────── Command (blhost-script) help ───────────────────────
Export the configuration option words to blhost script.
Usage: nxpmemcfg blhost-script [OPTIONS]

Options:
  Known chip select. Alternative to known chip is YAML configuration '-c': [all_or_none]
    -f, --family [mimxrt798s|mimxrt595s|mimxrt1172|mimxrt1046..., and more. Use 'get-families' command to show all.]
                                  Select the chip family.
    -p, --peripheral [flexspi_nor|xspi_nor|flexspi_nand|semc_nor|semc_nand|spi_nor|mmc|sd]
                                  Choose the peripheral of the input o

In [7]:
# Generate script to configure FlexSPI interface for the specific memory and extract FCB.
%! nxpmemcfg blhost-script -f mimxrt1189 -m W25QxxxJV -i quad_spi -p flexspi_nor -ix 1 -o $CFG_MEM_FILE --force --fcb $FCB_FILE

assert os.path.exists(CFG_MEM_FILE)

nxpmemcfg blhost-script -f mimxrt1189 -m W25QxxxJV -i quad_spi -p flexspi_nor -ix 1 -o workspace/config_mem.bls --force --fcb workspace/fcb.bin 
Loaded option words: Opt0: 0xC0000007
Exported blhost script.


### EVK Board Overview
#### The following picture describes connector placement of RT1180 EVK.
![Signing key and used SRK definition](img/evk-1.png "signing key")

#### Serial Downloader Protocol Mode:

Configure Boot Mode Switch to : 1/2/3-OFF, 4-ON\
Connect micro USB cable into USB OTG1         - J33\
Connect micro USB cable into Debug USB Port   - J53

#### Internal Boot Mode:

Configure Boot Mode Switch to :1-OFF, 2-ON, 3-OFF, 4-OFF\
Connect micro USB cable into Debug USB Port - J53

In [8]:
# The board must be in SDP mode
# Configure Boot Mode Switch to : 1/2/3-OFF, 4-ON
# Connect micro USB cable into USB OTG1         - J33
# Connect micro USB cable into Debug USB Port   - J53

# Parameters for BLHOST. USB PID/VID for ROM Flashloader and NXP Flashloader

COMPAR = "-u"  # USB
BLHOST_CONNECT_ROM = "0x1fc9,0x014c"  # PID/VID of ROM
BLHOST_CONNECT_FLDR = "0x15A2,0x0073"  # PID/VID of NXP Flashloader

# To use tha UART compar parameter must be change, COMxx must be replaced by the name of your COM port, 115200 is default baudrate.
# compar="-p"                                                           # UART
# BLHOST_CONNECT_ROM="COMxx,115200"                                     # COM port, baudrate
# BLHOST_CONNECT_FLDR="COMxx,115200"                                    # COM port, baudrate

assert os.path.exists(FLASHLOADER_FILE)

# Load flashloader
%! blhost $VERBOSITY $COMPAR $BLHOST_CONNECT_ROM get-property 1
%! blhost $VERBOSITY $COMPAR $BLHOST_CONNECT_ROM load-image $FLASHLOADER_FILE

blhost  -u 0x1fc9,0x014c get-property 1 
Response status = 0 (0x0) Success.
Response word 1 = 1258487809 (0x4b030001)
Current Version = K3.0.1
blhost  -u 0x1fc9,0x014c load-image ../rt118x_signed_flashloader/workspace/flashloader.bin 
Loading image
Response status = 0 (0x0) Success.


In [9]:
# Ping the RT1180
%! blhost $VERBOSITY $COMPAR $BLHOST_CONNECT_FLDR get-property 7
# Ping the RT1180
%! blhost $VERBOSITY $COMPAR $BLHOST_CONNECT_FLDR get-property 1

# Execute the commands generated by "blhost-script". This script will configure flash,
# program FCB into the external memory and reads back the binary data. Be aware tha erase of the first sector must be done.
# The script can be executed only once to read FCB.
%! blhost $COMPAR $BLHOST_CONNECT_FLDR batch $CFG_MEM_FILE
%! blhost $COMPAR $BLHOST_CONNECT_FLDR list-memory

assert os.path.exists(FCB_FILE)

blhost  -u 0x15A2,0x0073 get-property 7 
Response status = 0 (0x0) Success.
Response word 1 = 25530335 (0x1858fdf)
Available Commands = ['FlashEraseAll', 'FlashEraseRegion', 'ReadMemory', 'WriteMemory', 'FillMemory', 'GetProperty', 'ReceiveSBFile', 'Execute', 'Call', 'Reset', 'SetProperty', 'FlashReadResource', 'ConfigureMemory', 'GenerateKeyBlob', 'UpdateLifeCycle', 'EleMessage']
blhost  -u 0x15A2,0x0073 get-property 1 
Response status = 0 (0x0) Success.
Response word 1 = 1258424320 (0x4b020800)
Current Version = K2.8.0
blhost -u 0x15A2,0x0073 batch workspace/config_mem.bls 
Response status = 0 (0x0) Success.
Response status = 0 (0x0) Success.
Response status = 0 (0x0) Success.
Response status = 0 (0x0) Success.
Response status = 0 (0x0) Success.
Response status = 0 (0x0) Success.
Response status = 0 (0x0) Success.
Reading memory
Response status = 0 (0x0) Success.
Response word 1 = 512 (0x200)
Read 512 of 512 bytes.
blhost -u 0x15A2,0x0073 list-memory 
Internal Flash:
Internal RAM:
  

### 5.1 Create the bootable image

In [10]:
YamlDiffWidget("inputs/rt118x_secure_boot_bimg.diffc").html

nxpimage bootable-image get-templates -f mimxrt1189 -o workspace --force 
Creating workspace/bootimg_mimxrt1189_serial_downloader.yaml template file.
Creating workspace/bootimg_mimxrt1189_flexspi_nor.yaml template file.
Creating workspace/bootimg_mimxrt1189_flexspi_nand.yaml template file.
Creating workspace/bootimg_mimxrt1189_semc_nand.yaml template file.
Creating workspace/bootimg_mimxrt1189_emmc.yaml template file.
Creating workspace/bootimg_mimxrt1189_sd.yaml template file.


Export the final bootable image

In [11]:
# Export bootable image
%! nxpimage $VERBOSITY bootable-image export -c $BIMG_FILE -o $BOOTABLE_IMAGE

# Assert os.path.exists(BOOTABLE_IMAGE)
assert os.path.isfile(BOOTABLE_IMAGE)

nxpimage  bootable-image export -c inputs/bootimg_rt118x_flexspi_nor.yaml -o workspace/bootable_img.bin 
Success. (Bootable Image: workspace/bootable_img.bin created) 


### 5.2 Program the image into external memory

In [12]:
# Base address of FlexSPI 1
FLASH_ADDRESS = "0x28000000"
# FLASH_ADDRESS = "0x38000000"  # Secure alias

# Base address of FlexSPI 2
# FLASH_ADDRESS = "0x04000000"

# Size of external memory to erase
ERASE_SIZE = "0x50000"

# Erase flash
%! blhost $VERBOSITY $COMPAR $BLHOST_CONNECT_FLDR flash-erase-region $FLASH_ADDRESS $ERASE_SIZE
# Write image - 0x38000000
%! blhost $VERBOSITY $COMPAR $BLHOST_CONNECT_FLDR write-memory $FLASH_ADDRESS $BOOTABLE_IMAGE

# Switch the board into internal boot and reset.

# Configure Boot Mode Switch to :1-OFF, 2-ON, 3-OFF, 4-OFF
# Connect micro USB cable into Debug USB Port - J53
# Open Serial Terminal (115200)
# Reset the board

# The "hello_world" with time should be printed on the terminal:
# hello world new 14:36:56

blhost -u 0x15A2,0x0073 flash-erase-region 0x28000000 0x50000 
Response status = 0 (0x0) Success.
blhost -u 0x15A2,0x0073 write-memory 0x28000000 workspace/bootable_img.bin 
Writing memory
Response status = 0 (0x0) Success.
Response word 1 = 61440 (0xf000)


## 6. Burn Super Root Key Hash (SRKH) and Manage Lifecycle.
After successful validation of previous steps we can proceed with final security enablement. SRKH must be programmed into corresponding fuses to establish root of trust. On RT118x the dedicated ELE command must be used to move lifecycle to OEM_CLOSED which enforces security.

<center><span style="color:red">NOTE!</span></center>
<span style="color:red">Device will be put into OEM_CLOSED Lifecycle after execution of the scrip below. It becomes mandatory to sign all FW running on the device by corresponding private key.</span>

<span style="color:red">Only authenticated FW images will be executed. Debug features will be limited and must be unlocked by debug mailbox feature.</span> 

Debug mailbox example is located [HERE](../../dat/mimxrt1189/rt118x_debug_authentication.ipynb).

-	WARNING!!! This step is destructive operation (updating life cycle)
```
# The board must be in SDP mode
# Configure Boot Mode Switch to : 1/2/3-OFF, 4-ON
# Connect micro USB cable into USB OTG1         - J33
# Connect micro USB cable into Debug USB Port   - J53

from time import sleep

assert os.path.exists(FLASHLOADER_FILE)
assert os.path.exists(SRKH_BATCH_FILE)

# To enable this script, set the variable below to 1
enable_script = 0
if enable_script:

    # Ping ROM flashloader
    %! blhost $VERBOSITY $COMPAR $BLHOST_CONNECT_ROM get-property 1
    # Load flashloader
    %! blhost $VERBOSITY $COMPAR $BLHOST_CONNECT_ROM load-image $FLASHLOADER_FILE
    sleep(3)
    # Ping flashloader
    %! blhost $VERBOSITY $COMPAR $BLHOST_CONNECT_FLDR get-property 1
    # Execute batch command to program SRKH
    %! blhost $VERBOSITY $COMPAR $BLHOST_CONNECT_FLDR batch $SRKH_BATCH_FILE

    # OEM lifecycle statuses
    # 8 - oem_closed: Update to OEM Closed
    ELE_LIFECYCLE = 8

    # Change Lifecycle to OEM_CLOSED
    %! blhost $VERBOSITY $COMPAR $BLHOST_CONNECT_FLDR update-life-cycle $ELE_LIFECYCLE

# Switch the board into internal boot and reset.

# Configure Boot Mode Switch to :1-OFF, 2-ON, 3-OFF, 4-OFF
# Connect micro USB cable into Debug USB Port - J53
# Open Serial Terminal (115200)
# Reset the board

# The "hello_world" with time should be printed on the terminal:
# hello world new 14:36:56
```